In [90]:
import numpy as np
import modin.pandas as pd
import quandl as qd
import math
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import LinearRegression

In [91]:
qd.ApiConfig.api_key = 'jNYadxyk6igH2w32z-a_'

In [92]:
df=qd.get('WIKI/GOOGL')

In [93]:
df

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-21,1092.57,1108.70,1087.21,1094.000,1990515.0,0.0,1.0,1092.570000,1108.700000,1087.210000,1094.000000,1990515.0
2018-03-22,1080.01,1083.92,1049.64,1053.150,3418154.0,0.0,1.0,1080.010000,1083.920000,1049.640000,1053.150000,3418154.0
2018-03-23,1051.37,1066.78,1024.87,1026.550,2413517.0,0.0,1.0,1051.370000,1066.780000,1024.870000,1026.550000,2413517.0


In [94]:
df[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']].tail(20)

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,
2018-02-28,1122.00,1127.65,1103.00,1103.92,2431023.0
2018-03-01,1109.54,1111.27,1067.29,1071.41,2766856.0
2018-03-02,1057.98,1086.89,1050.11,1084.14,2508145.0
2018-03-05,1078.13,1101.18,1072.27,1094.76,1432369.0
2018-03-06,1102.10,1105.63,1094.50,1100.90,1169068.0
2018-03-07,1092.82,1116.20,1089.91,1115.04,1537429.0
2018-03-08,1117.20,1131.44,1117.20,1129.38,1510478.0
2018-03-09,1139.50,1161.00,1134.29,1160.84,2070174.0
2018-03-12,1165.05,1178.16,1159.20,1165.93,2129297.0


In [95]:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close']*100.0

In [96]:
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open']*100.0

In [97]:
df=df[['Adj. Close','HL_PCT', 'PCT_change', 'Adj. Volume']]

In [98]:
df.tail(10)

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2018-03-14,1148.89,0.946131,0.269681,2033697.0
2018-03-15,1150.61,1.033365,0.090469,1623868.0
2018-03-16,1134.42,1.973696,-1.811572,2654602.0
2018-03-19,1100.07,1.754434,-1.582630,3076349.0
2018-03-20,1095.80,0.889761,-0.236708,2709310.0
2018-03-21,1094.00,1.343693,0.130884,1990515.0
2018-03-22,1053.15,2.921711,-2.487014,3418154.0
2018-03-23,1026.55,3.918952,-2.360729,2413517.0
2018-03-26,1054.09,0.491419,0.332191,3272409.0


In [99]:
forecast_col = "Adj. Close"
df.fillna(-99999, inplace=True)

In [100]:
forecast_out = int(math.ceil(0.1*len(df)))

In [101]:
df['label']= df[forecast_col].shift(-forecast_out)

In [102]:
df.dropna(inplace=True)

In [103]:
X = np.array(df.drop(['label'],1))
Y = np.array(df['label'])

In [104]:
X = preprocessing.scale(X)

In [105]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.8)
clf = LinearRegression()
clf.fit(X_train,Y_train)
model_accuracy= clf.score(X_test, Y_test)
model_accuracy

0.8807589752919109

In [110]:
clf = svm.SVR()
clf.fit(X_train,Y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [112]:
model_accuracy= clf.score(X_test,Y_test)
model_accuracy

0.29119637153138467